In [9]:
import json

from datasets import Dataset
from transformers import DataCollatorForTokenClassification
from extr_ds.manager.utils.filesystem import load_document
import numpy
import evaluate
from transformers.keras_callbacks import KerasMetricCallback
import tensorflow as tf
from transformers import AutoTokenizer, \
    TFAutoModelForTokenClassification
from transformers import pipeline

import warnings
warnings.filterwarnings('ignore')

In [13]:
epochs = 3
model_checkpoint = 'bert-base-cased'
model_output_checkpoint = 'transformers/nfl_pbp_token_classifier'

labels = [
    "abbr",
    "orth",
    "MorfDef",
    "gramGrp",
    "label",
    "citRange",
    "quote",
    "citedRange",
    "hi",
    "RegDef",
    "def",
    "cit",
    "term",
    "norm",
    "form",
    "bibl",
    "usg"
]

label2id = {label: i for i, label in enumerate(labels)}
id2label = {i: label for i, label in enumerate(labels)}

In [14]:
def align_labels(tokenized_inputs, label_list):
    labels = []
    for word_idx in tokenized_inputs.word_ids(batch_index=0):
        label_id = -100
        if word_idx is not None:
            label = label_list[word_idx]

            label_id = label2id[label]

        labels.append(label_id)
        # previous_word_idx = word_idx

    return labels


def get_dataset(tokenizer, model):
    def tokenize_and_align_labels(record):
        tokenized_inputs = tokenizer(
            record['tokens'],
            truncation=True,
            is_split_into_words=True
        )

        tokenized_inputs['labels'] = align_labels(
            tokenized_inputs,
            record['labels']
        )

        return tokenized_inputs

    ents_dataset = json.loads(
        load_document('dataset/dataset.json')
    )

    # random.shuffle(ents_dataset)

    pivot = int(len(ents_dataset) * .8)
    data_collator = DataCollatorForTokenClassification(
        tokenizer,
        return_tensors='tf'
    )

    train_dataset = Dataset.from_list(ents_dataset[:pivot])
    tf_train_set = model.prepare_tf_dataset(
        train_dataset.map(
            tokenize_and_align_labels,
            batched=False
        ),
        shuffle=True,
        collate_fn=data_collator,
    )

    test_dataset = Dataset.from_list(ents_dataset[pivot:])
    tf_test_set = model.prepare_tf_dataset(
        test_dataset.map(
            tokenize_and_align_labels,
            batched=False
        ),
        shuffle=True,
        collate_fn=data_collator,
    )

    return tf_train_set, tf_test_set


seqeval = evaluate.load('seqeval')


def compute_metrics(preds):
    predictions, actuals = preds
    predictions = numpy.argmax(predictions, axis=2)

    results = seqeval.compute(
        predictions=[
            [labels[p] for p, l in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, actuals)
        ],
        references=[
            [labels[l] for p, l in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, actuals)
        ]
    )

    return {
        key: results[f'overall_{key}']
        for key in ['precision', 'recall', 'f1', 'accuracy']
    }

In [15]:
tokenizer = AutoTokenizer.from_pretrained(
    model_checkpoint
)

model = TFAutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(labels),
    id2label=id2label,
    label2id=label2id
)

tf_train_set, tf_test_set = get_dataset(tokenizer, model)

callbacks = [
    KerasMetricCallback(
        metric_fn=compute_metrics,
        eval_dataset=tf_test_set
    ),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
]

optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
model.compile(optimizer=optimizer)

All PyTorch model weights were used when initializing TFBertForTokenClassification.

Some weights or buffers of the TF 2.0 model TFBertForTokenClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/529 [00:00<?, ? examples/s]

Map:   0%|          | 0/133 [00:00<?, ? examples/s]

In [ ]:
model.fit(
    x=tf_train_set,
    validation_data=tf_test_set,
    epochs=epochs,
    callbacks=callbacks
)

Epoch 1/10
66/66 [==============================] - 1276s 19s/step - loss: 1.3206 - val_loss: 1.6987 - precision: 0.2136 - recall: 0.2864 - f1: 0.2447 - accuracy: 0.3733
Epoch 2/10
66/66 [==============================] - 1258s 19s/step - loss: 0.4523 - val_loss: 2.0068 - precision: 0.3189 - recall: 0.3717 - f1: 0.3433 - accuracy: 0.3986
Epoch 3/10
66/66 [==============================] - 1355s 21s/step - loss: 0.2961 - val_loss: 1.2717 - precision: 0.2668 - recall: 0.3779 - f1: 0.3128 - accuracy: 0.4453
Epoch 4/10
66/66 [==============================] - 1294s 20s/step - loss: 0.1862 - val_loss: 1.2803 - precision: 0.2330 - recall: 0.3818 - f1: 0.2894 - accuracy: 0.5391
Epoch 5/10
66/66 [==============================] - 1297s 19s/step - loss: 0.1281 - val_loss: 2.0234 - precision: 0.3567 - recall: 0.4251 - f1: 0.3879 - accuracy: 0.4118
Epoch 6/10
18/66 [=======>......................] - ETA: 11:39 - loss: 0.1009

In [11]:
for model_to_save in [tokenizer, model]:
  model_to_save.save_pretrained(model_output_checkpoint)

In [12]:
classifier = pipeline(
    'ner',
    model=model_output_checkpoint,
    aggregation_strategy='simple'
)

examples = [
  'DISFAVORITOR, -OÂRE adj. (învechit, rar) Care defavorizează. A tractarisi şi a-şi dobândi oarecare tocmele mai puţin disfavoritoare. AR (1829), 1071/34.     - Pl.: disfavoritori, -oare. — De la disfavoare.',
]

responses = classifier(examples)
print(responses)

Some layers from the model checkpoint at transformers/nfl_pbp_token_classifier were not used when initializing TFBertForTokenClassification: ['dropout_75']
- This IS expected if you are initializing TFBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForTokenClassification were initialized from the model checkpoint at transformers/nfl_pbp_token_classifier.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForTokenClassification for predictions without further training.


[[{'entity_group': 'orth', 'score': 0.9527491, 'word': 'DISFAVORITOR, - OÂRE', 'start': 0, 'end': 19}, {'entity_group': 'gramGrp', 'score': 0.672104, 'word': 'adj', 'start': 20, 'end': 23}, {'entity_group': 'MorfDef', 'score': 0.64761406, 'word': '.', 'start': 23, 'end': 24}, {'entity_group': 'usg', 'score': 0.8015866, 'word': '( învechit, rar )', 'start': 25, 'end': 40}, {'entity_group': 'RegDef', 'score': 0.8467941, 'word': 'Care defavorizează.', 'start': 41, 'end': 60}, {'entity_group': 'hi', 'score': 0.5822195, 'word': 'A', 'start': 61, 'end': 62}, {'entity_group': 'RegDef', 'score': 0.41776785, 'word': 'tract', 'start': 63, 'end': 68}, {'entity_group': 'hi', 'score': 0.570468, 'word': '##arisi şi a - şi dobândi oarecare tocmele mai puţin disfavoritoare.', 'start': 68, 'end': 132}, {'entity_group': 'abbr', 'score': 0.7345371, 'word': 'AR ( 1829 ), 1071 / 34.', 'start': 133, 'end': 152}, {'entity_group': 'form', 'score': 0.76842046, 'word': '- Pl. :', 'start': 157, 'end': 163}, {'en